In [ ]:
import psycopg2
import pandas as pd

from sqlalchemy import create_engine, text as sql_text


# Connection details
host_p='trumpet.db.elephantsql.com'
database_p='dvbavdzj'
user_p='dvbavdzj'
password_p='r_iwQfAkOG8x-FIPbB44eCgLIByVjXsx'
port_p = 5432


# #establishing the connection
conn = psycopg2.connect(database=database_p, user=user_p, password=password_p, host=host_p, port=port_p)
#Creating a cursor object using the cursor() method
conn.autocommit = True
cursor = conn.cursor()


connection_string = f'postgresql://{user_p}:{password_p}@{host_p}:{port_p}/{database_p}'
engine_postgresql = create_engine(connection_string)

# 1
To find out BeeCycle's target customers. You perform a query to see the distribution of customers based on gender and age.

Where for age categories, you divide the customer's age into (Hint: CASE WHEN)

customer age <= 20 years then 'Group <=20' customer age between 21 and 40 years then 'Group 21 - 40' customer age between 41 and 60 years then 'Group 41 - 60' customer age more than 60 years then 'Group >60' With the grouping_age that has been defined, then answer the following question

What age grouping and what gender has the highest transactions on BeeCycle

In [ ]:
q = '''
    select
    case
      when age <= 20 then 'Group <= 20'
      when age between 20 and 40 then 'Group 20 - 40'
      when age between 41 and 60 then 'Group 41 - 60'
      when age > 60 then 'Group > 60'
    end as grouping_age,
        gender,
        sum (totalprice_rupiah) as total_transactions
    from fact_sales fs
      join dim_customer dc on fs.customer_id = dc.customer_id
    group by age, gender
    order by total_transactions desc



'''

df = pd.read_sql_query(con=engine_postgresql.connect(), sql=sql_text(q))
print (df.shape)
df

(98, 3)


,grouping_age,gender,total_transactions
0,Group 20 - 40,F,2.281104e+09
1,Group 20 - 40,M,2.192182e+09
2,Group 41 - 60,F,2.141420e+09
3,Group 20 - 40,M,1.883389e+09
4,Group 20 - 40,F,1.847363e+09
...,...,...,...
93,Group > 60,M,3.551660e+06
94,Group > 60,F,9.797200e+05
95,Group > 60,M,8.328600e+05
96,Group > 60,F,6.239800e+05


# 2
Because next October there will be a campaign. You have an idea to highlight the color of certain products that customers like. You will perform a query to answer the following questions:

What color in **every year** is the **most popular color bought** by customers?

In [ ]:
q = '''
    with data_rank as (
      select
        date_part('year', date(order_date)) as year,
        fs.product_id,
        product_name,
        color,
        count(distinct order_detail_id) as sales,
        row_number() over (partition by date_part('year', date(order_date)) order by count(distinct order_detail_id) desc) as rank
      from fact_sales fs
      join dim_product dp on fs.product_id = dp.product_id
      where color != 'NA'
      group by 1,2,3,4
      order by 1,4 desc
    )
    select * from data_rank
    where rank = 1;
'''

df = pd.read_sql_query(con=engine_postgresql.connect(), sql=sql_text(q))
print (df.shape)
df

(4, 6)


,year,product_id,product_name,color,sales,rank
0,2016.0,312,"Road-150 Red, 48",Red,48,1
1,2017.0,310,"Road-150 Red, 62",Red,45,1
2,2018.0,222,"Sport-100 Helmet, Blue",Blue,128,1
3,2019.0,222,"Sport-100 Helmet, Blue",Blue,93,1


# 3
Because the October campaign is only limited to each branch store. do you have an idea, how to only take the TOP 10 product_ids from each branch store. so you do a query to answer the following questions:

TOP 10 product names which are the most popular in each territory

In [ ]:
q='''
    select
      *
    from dim_territory
'''
df = pd.read_sql_query(con=engine_postgresql.connect(), sql=sql_text(q))
df

,territory_id,region,country,groups
0,1,Northwest,United States,North America
1,2,Northeast,United States,North America
2,3,Central,United States,North America
3,4,Southwest,United States,North America
4,5,Southeast,United States,North America
5,6,Canada,Canada,North America
6,7,France,France,Europe
7,8,Germany,Germany,Europe
8,9,Australia,Australia,Pacific
9,10,United Kingdom,United Kingdom,Europe


In [ ]:
q = '''
    with data_rank as (
      select
        fs.product_id,
        product_name,
        dt.territory_id,
        count(distinct order_detail_id) as sales,
        row_number() over (partition by dt.territory_id order by count(distinct order_detail_id) desc) as rank
      from fact_sales fs
      join dim_product dp on fs.product_id = dp.product_id
      join dim_territory dt on fs.territory_id = dt.territory_id
      group by 1,2,3
      order by 3,4 desc
    )
    select * from data_rank
    where rank <= 10;
'''

df = pd.read_sql_query(con=engine_postgresql.connect(), sql=sql_text(q))
print (df.shape)
df

(75, 5)


,product_id,product_name,territory_id,sales,rank
0,537,HL Mountain Tire,1,39,1
1,480,Patch Kit/8 Patches,1,34,2
2,528,Mountain Tire Tube,1,28,3
3,310,"Road-150 Red, 62",1,26,4
4,312,"Road-150 Red, 48",1,21,5
...,...,...,...,...,...
70,479,Road Bottle Cage,10,36,6
71,217,"Sport-100 Helmet, Black",10,23,7
72,541,Touring Tire,10,20,8
73,361,"Mountain-200 Black, 42",10,19,9


# 4
Because you have passed probation... you are asked to make your own analysis for the boss. Some steps that need to be taken are:

Create an objective analysis (example: Because the next campaign wants to be maximal, we will highlight the top 2 products)
Create a business question (example: What are the most popular products in terms of number of transactions?)
Build the Query to answer the question in point 2
Write Actionable knowledge or next action after getting the answer to point 3 (example: product abc is the highest so we will focus on promoting that product)

In [ ]:
q = '''
    with data_rank as (
      select
        fs.product_id,
        product_name,
        count(distinct order_detail_id) as sales,
        row_number() over (order by count(distinct order_detail_id) desc) as rank
      from fact_sales fs
      join dim_product dp on fs.product_id = dp.product_id
      group by 1,2
      order by 3 desc
    )
    select * from data_rank
    where rank <= 10;
'''

df = pd.read_sql_query(con=engine_postgresql.connect(), sql=sql_text(q))
print (df.shape)
df

(10, 4)


,product_id,product_name,sales,rank
0,477,Water Bottle - 30 oz.,363,1
1,480,Patch Kit/8 Patches,262,2
2,478,Mountain Bottle Cage,228,3
3,222,"Sport-100 Helmet, Blue",221,4
4,214,"Sport-100 Helmet, Red",201,5
5,528,Mountain Tire Tube,200,6
6,537,HL Mountain Tire,199,7
7,479,Road Bottle Cage,191,8
8,225,AWC Logo Cap,190,9
9,485,Fender Set - Mountain,188,10


### Objective Analysis : Because the next campaign wants to be maximal, we will find out who the customers are who rarely shop at this shop.

### Business Question : Who are the customers with low transaction frequency?

In [ ]:
q = '''
    select
      *
    from dim_customer

'''

df = pd.read_sql_query(con=engine_postgresql.connect(), sql=sql_text(q))
print (df.shape)
df

(1360, 9)


,customer_id,geography_id,customer_name,birthdate,maritalstatus,gender,datefirstpurchase,age,grouping_age
0,11013,543,Ian Jenkins,1988-08-06,M,M,2018-10-15,34,Group 21 - 40
1,11014,634,Sydney Bennett,1988-05-09,S,F,2018-09-24,34,Group 21 - 40
2,11015,301,Chloe Young,1999-02-27,S,F,2018-07-22,23,Group 21 - 40
3,11016,329,Wyatt Hill,1999-04-28,M,M,2018-08-13,23,Group 21 - 40
4,11017,39,Shannon Wang,1964-06-26,S,F,2016-07-15,58,Group 41 - 60
...,...,...,...,...,...,...,...,...,...
1355,28393,216,Ramon Zheng,1986-07-07,S,M,2016-07-30,36,Group 21 - 40
1356,29167,611,Dawn Shen,1979-03-12,S,F,2017-01-04,43,Group 41 - 60
1357,99997,37,Siska Budi,1996-10-29,S,F,None,26,Group 21 - 40
1358,99998,26,Diana Saraswati,1997-09-26,S,F,None,25,Group 21 - 40


In [ ]:
q = '''
    with data_rank as (
      select
        fs.customer_id,
        customer_name,
        count(distinct order_detail_id) as order,
        row_number() over (order by count(distinct order_detail_id) desc) as rank
      from fact_sales fs
      join dim_customer dc on fs.customer_id = dc.customer_id
      group by 1,2
      order by 3 asc
    )
    select * from data_rank
'''

df = pd.read_sql_query(con=engine_postgresql.connect(), sql=sql_text(q))
print (df.shape)
df

(1357, 4)


,customer_id,customer_name,order,rank
0,28659,Vanessa Simmons,1,1216
1,14214,Robyn Munoz,1,1357
2,11533,Ebony Gill,1,1356
3,29281,Charles Anderson,1,1355
4,14312,Claudia Lin,1,1354
...,...,...,...,...
1352,11566,April Shan,35,5
1353,11277,Charles Jackson,38,4
1354,11091,Dalton Perez,38,3
1355,11287,Henry Garcia,41,2


### Next Action : After finding out which customers rarely shop at this shop in terms of transaction frequency, we will focus on providing attractive promotions for customers, especially those with low transaction frequency, so they will shop/make transactions again at this store